## Data Analysis Protocol

**Remove Header Information as it interferes with reading of CSV files**

**Import Libraries:** Import the necessary libraries.

**Get the List of Files:** You can use the os module to get a list of files in a directory.

**Read and Append CSV Files:** Use a loop to read each CSV file and append it to a DataFrame.

In [1]:
#import modules
import os
import glob
import pandas as pd
import numpy as np


In [2]:
path = "directory/"
file_list = os.listdir(path)
file_type = (file_list[0])[-3:]
print(f'Found {len(file_list)} {file_type} files.')

Found 155 csv files.


Get the list of CSV files in the directory


In [3]:
def csvheaders(path:str):
    file_list = os.listdir(path)
    csvobj=path+file_list[0]
    a = pd.DataFrame()
    a = pd.read_csv(csvobj,sep=';',skiprows=14, na_values=['N/A'])
    df_headers = a.columns.values

    counter=-1
    for header in df_headers:
        counter+=1
        print(f'Index: {counter} - {header}')
    return df_headers

directory_headers = csvheaders("directory/")


Index: 0 - Folder
Index: 1 - Filename
Index: 2 - SerialNumber
Index: 3 - FileCode
Index: 4 - DownloadIdentifier
Index: 5 - DayOfWeek
Index: 6 - Date
Index: 7 - ValidDay
Index: 8 - TotalTime(m)
Index: 9 - TotalRLMsCount
Index: 10 - StepCount
Index: 11 - CyclingStepCount
Index: 12 - TotalRLMsTime(m)
Index: 13 - SteppingTime(m)
Index: 14 - CyclingTime(m)
Index: 15 - StandingTime(m)
Index: 16 - TotalSedentaryTime(m)
Index: 17 - SittingTime(m)
Index: 18 - SeatedTransportTime(m)
Index: 19 - SecondaryLyingTime(m)
Index: 20 - PrimaryLyingTime(m)
Index: 21 - NonWearTime(m)
Index: 22 - ActivityScore(MET.h)
Index: 23 - NumSitToStands
Index: 24 - ProportionOfRLMsBouts<1m(%)
Index: 25 - RLMsCountInBouts(>=10s,<1m)
Index: 26 - RLMsCountInBouts(>=1m,<5m)
Index: 27 - RLMsCountInBouts(>=5m,<10m)
Index: 28 - RLMsCountInBouts(>=10m,<20m)
Index: 29 - RLMsCountInBouts(>=20m)
Index: 30 - TimeInRLMsBouts(m)(>=10s,<1m)
Index: 31 - TimeInRLMsBouts(m)(>=1m,<5m)
Index: 32 - TimeInRLMsBouts(m)(>=5m,<10m)
Index: 3

In [19]:
df_headers = ["Identifier","Average Step Count","Average Step Time","Average Stand Count","Average Sitting Time","Average Sedentary Time","NumSitToStands"]
print(df_headers)

['Identifier', 'Average Step Count', 'Average Step Time', 'Average Stand Count', 'Average Sitting Time', 'Average Sedentary Time', 'NumSitToStands']


In [4]:
# path = "directory/"
# file_list = os.listdir(path)
# for file in file_list:
#     df_temp = pd.read_csv(path+file,sep=';',skiprows=15, na_values=['N/A'],header=0)
#     #Reads the 5th to 7th characters in the file name to identify if it is BL PI or FU
#     if file[4:6] == "BL":
#         #Sort to BL folder if BL
#         df_temp.to_csv(os.path.join("timepoints/BL/", os.path.basename(file)), na_rep='N/A', index=False)
#     elif (str(file[4:6]) == "PI"):
#         #Sort to PI folder if PI
#         df_temp.to_csv(os.path.join("timepoints/PI/", os.path.basename(file)), na_rep='N/A', index=False)
#     elif (str(file[4:6]) == "FU"):
#         #Sort to FU folder if FU
#         df_temp.to_csv(os.path.join("timepoints/FU/", os.path.basename(file)), na_rep='N/A', index=False)
#     else:
#         print(f"error on {file}")

In [22]:
def ActivPalDataProcessor(path): 
    index=0
    file_list = os.listdir(path)
    summarydata = []
    for file in file_list:
        try:
            df_temp = pd.read_csv(path+file,na_values=['N/A'],header=0)
            array = df_temp.to_numpy()
            avgstepcount = np.empty(array.shape[0])
            avgsteptime = np.empty(array.shape[0])
            avgstandtime = np.empty(array.shape[0])
            avgsittime = np.empty(array.shape[0])
            avgsedtime = np.empty(array.shape[0])
            NSTS = np.empty(array.shape[0])
            for i in range (array.shape[0]):
                data = array[i]
                participantid = str(data[1])[:3]
                avgstepcount[i] = (data[10])
                avgsteptime[i] = (data[13])
                avgstandtime[i] = (data[15])
                avgsittime[i] = (data[17])
                avgsedtime[i] = (data[16])
                NSTS[i] = (data[23])

            summarydata += [[participantid,avgstepcount.mean(),avgsteptime.mean(),avgstandtime.mean(),avgsittime.mean(),avgsedtime.mean(),NSTS.mean()]]
            index += 1
        except IndexError:
            # summarydata += [[participantid,avgstepcount.mean(),avgsteptime.mean(),avgstandtime.mean(),avgsittime.mean(),avgsedtime.mean(),NSTS.mean()]]
            print(f"NaN IndexNo.{index} Error{[[participantid,avgstepcount.mean(),avgsteptime.mean(),avgstandtime.mean(),avgsittime.mean(),avgsedtime.mean(),NSTS.mean()]]}")
            print('---------')
            print("") 
    return summarydata , pd.DataFrame(summarydata,columns=df_headers)

In [23]:
BLDS , BLPD = ActivPalDataProcessor("timepoints/BL/")
BLPD.to_csv("Baseline.csv")

In [24]:
PIDS,PIDP= ActivPalDataProcessor("timepoints/PI/")
PIDP.to_csv("PostInvervention.csv")

In [25]:
FUDS,FUDP = ActivPalDataProcessor("timepoints/FU/")
FUDP.to_csv("FollowUp.csv")

In [99]:

bl= []
pi= [] 
fu= [] 
def survey_data(array):
    seen = 0 # create an empty set
    bl= []
    pi= [] 
    fu= []
    try:
        for entry in array:
            seen += 1
            if entry in bl :
                if entry in pi:
                    if entry in fu:
                        print(f"Extra Participant ID#: {entry} index #:{seen} ")
                    else:
                        fu += [entry]
                else:
                    pi += [entry]
            else:
                bl += [entry] # add entry to the set
                # do something with entry
        print("----------------------------")
        print(f'Total: {len(bl+pi+fu)} Baseline: {len(bl)} PostIntervention: {len(pi)} FollowUp: {len(fu)}')
        print("----------------------------")

        print("PI but not in FU")
        print("----------------------------")

        for entry in pi:
            if entry in fu:
                pass
            else:
                print(f"(FU-/PI+)Missing Participant ID#: {entry}")
    
        for entry in fu:
            if entry in pi:
                pass
            else:
                print(f"(FU+/PI-)Missing Participant ID#: {entry}")
        for entry in bl:
            if entry in pi:
                pass
            else:
                print(f"(BL+/PI-)Missing Participant ID#: {entry}")
    except:
        print("An error occurred")
    return bl,pi,fu

bl,pi,fu=survey_data(array)

Extra Participant ID#: 66 index #:142 
----------------------------
Total: 147 Baseline: 54 PostIntervention: 48 FollowUp: 45
----------------------------
PI but not in FU
----------------------------
(FU-/PI+)Missing Participant ID#: 10
(FU-/PI+)Missing Participant ID#: 9
(FU-/PI+)Missing Participant ID#: 49
(BL+/PI-)Missing Participant ID#: 1
(BL+/PI-)Missing Participant ID#: 5
(BL+/PI-)Missing Participant ID#: 6
(BL+/PI-)Missing Participant ID#: 7
(BL+/PI-)Missing Participant ID#: 4
(BL+/PI-)Missing Participant ID#: 65


<!-- - Extra Participant ID#: 2 index #:20 
- Extra Participant ID#: 3 index #:21 
- Extra Participant ID#: 8 index #:22 
- Extra Participant ID#: 63 index #:150 
- Extra Participant ID#: 71 index #:152 

Total: 148 Baseline: 55 PostIntervention: 49 FollowUp: 44

PI but not in FU

- Missing Participant ID#: 1
- Missing Participant ID#: 10
- Missing Participant ID#: 9
- Missing Participant ID#: 49
- Missing Participant ID#: 70 -->

In [23]:
surveydata = pd.read_excel("surveydata.xlsx",skiprows=1,na_values=['N/A'],header=0)
sleepdf1=surveydata.to_numpy()
weekday_wake=[]
weekday_sleep=[]
weekend_wake=[]
weekend_sleep=[]
weekday_sleep_duration=[]
weekend_sleep_duration=[]
avg_sleep_duration = []

for i in range (sleepdf1.shape[0]):
    sleepdf=sleepdf1[i]
    weekday_wake += [sleepdf[11]]
    weekday_sleep +=[sleepdf[12]]
    weekend_wake += [sleepdf[13]]
    weekend_sleep += [sleepdf[14]] 
    weekday_sleep_duration +=  [pd.Timedelta((pd.to_datetime(sleepdf[12])-(pd.to_datetime(sleepdf[11])))).seconds/60]
    weekend_sleep_duration += [pd.Timedelta((pd.to_datetime(sleepdf[14])-(pd.to_datetime(sleepdf[13])))).seconds/60]
    avg_sleep_duration += [((pd.Timedelta((pd.to_datetime(sleepdf[12])-(pd.to_datetime(sleepdf[11])))).seconds/60)*(5/7)+((pd.Timedelta((pd.to_datetime(sleepdf[14])-(pd.to_datetime(sleepdf[13])))).seconds/60)*(2/7)))]
print(len(avg_sleep_duration))

In [34]:
for i in range (len(avg_sleep_duration)):
    print(f'Interview #: {index[i]} \ Participant {array[i]} \ wAvg: {avg_sleep_duration[i]} \ {weekday_sleep_duration[i]} + {weekend_sleep_duration[i]}')


Interview #: 3 \ Participant 1 \ wAvg: 482.1428571428571 \ 495.0 + 450.0
Interview #: 5 \ Participant 2 \ wAvg: 531.4285714285714 \ 510.0 + 585.0
Interview #: 6 \ Participant 5 \ wAvg: 338.57142857142856 \ 330.0 + 360.0
Interview #: 7 \ Participant 6 \ wAvg: 420.0 \ 420.0 + 420.0
Interview #: 8 \ Participant 7 \ wAvg: 447.8571428571429 \ 435.0 + 480.0
Interview #: 9 \ Participant 3 \ wAvg: 475.7142857142857 \ 450.0 + 540.0
Interview #: 10 \ Participant 8 \ wAvg: 411.42857142857144 \ 420.0 + 390.0
Interview #: 11 \ Participant 9 \ wAvg: 454.2857142857142 \ 390.0 + 615.0
Interview #: 12 \ Participant 4 \ wAvg: 413.57142857142856 \ 405.0 + 435.0
Interview #: 13 \ Participant 10 \ wAvg: 535.7142857142857 \ 510.0 + 600.0
Interview #: 14 \ Participant 1 \ wAvg: 447.8571428571429 \ 465.0 + 405.0
Interview #: 15 \ Participant 2 \ wAvg: 535.7142857142857 \ 510.0 + 600.0
Interview #: 17 \ Participant 3 \ wAvg: 477.8571428571429 \ 465.0 + 510.0
Interview #: 18 \ Participant 8 \ wAvg: 441.42857142

In [ ]:
FinalCSV = pd.DataFrame("npArray",columns=headers)

In [131]:
array=np.array(df_append, dtype=object)